# LangChain: Evaluation

Watch:

https://learn.deeplearning.ai/langchain/lesson/6/evaluation 

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [3]:
llm_model: str = "gpt-3.5-turbo-1106"

## Create our QandA application

In [4]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [5]:
from langchain.schema import Document

file: str = 'OutdoorClothingCatalog_1000.csv'
loader: CSVLoader = CSVLoader(file_path=file)
data: list[Document] = loader.load()

In [6]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

index: VectorStoreIndexWrapper = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-9oP4S86HpNNbJMudY026ckbW on tokens per min (TPM): Limit 150000, Used 130971, Requested 30073. Please try again in 4.417s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


In [7]:
from langchain.chains.retrieval_qa.base import BaseRetrievalQA

llm: ChatOpenAI = ChatOpenAI(temperature = 0.0, model=llm_model)
qa: BaseRetrievalQA = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [8]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10})

In [9]:
data[11]

Document(page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11})

### Hard-coded examples

In [10]:
examples: list[dict[str, str]] = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [11]:
from langchain.evaluation.qa import QAGenerateChain

In [12]:
example_gen_chain: QAGenerateChain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [21]:
# the warning below can be safely ignored

In [28]:

new_examples  = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/Users/ZiaKhan/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-1106 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min (RPM): Limit 3, Used 3, Reque

In [29]:
new_examples[0]

{'qa_pairs': {'query': "What are the key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The key features of the Women's Campside Oxfords as described in the document include ultracomfortable lace-to-toe Oxford design, super-soft canvas material for a broken-in feel and look, thick cushioning, quality construction, comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, vintage hunt, fish and camping motif on innersole, moderate arch contour of innersole, EVA foam midsole for cushioning and support, and chain-tread-inspired molded rubber outsole with modified chain-tread pattern."}}

In [30]:
data[0]


Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

### Combine examples

In [31]:
# https://www.codecademy.com/forum_questions/559a2e9576b8fec400000392

examples += new_examples


In [ ]:

qa.run(examples[0]["query"])

## Manual Evaluation

In [ ]:
import langchain
langchain.debug = True

In [ ]:
qa.run(examples[0]["query"])

In [ ]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [ ]:
predictions: list[dict[str, str]] = qa.apply(examples)

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm: ChatOpenAI = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs: list[dict[str, str]]  = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

In [ ]:
graded_outputs[0]